In [ ]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [ ]:
%%capture
!pip install datasets==2.1.0
!pip install transformers==4.18.0
!pip install librosa

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("superb", "ks")
metric = load_metric("accuracy")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6798 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3081 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

print(id2label["9"])
print(label2id["go"])

go
9


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [ ]:
max_duration = 1.0  # seconds

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [ ]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/52 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
# INTRODUCE TRAIN TEST VAL SPLITS
from datasets import DatasetDict, load_dataset, load_metric

# 90% train, 10% test + validation
train_testvalid = encoded_dataset["test"].train_test_split(shuffle=True, test_size=0.8)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.1)
test_valid_1 = test_valid['test'].train_test_split(test_size=0.1)

# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['test'],
    'test': test_valid_1['train']})

In [ ]:
print(ds['train']['label'][0])

9


In [ ]:
for i in range(len(ds['train']['input_values'])):
  if(ds['train']['label']==9):
    print(ds['train']['input_values'][i])


[-0.01767951063811779, -0.06257089972496033, -0.017913319170475006, 0.06158186495304108, 0.07981899380683899, -0.0006114261341281235, -0.10839753597974777, -0.11143705993890762, 0.047553300857543945, 0.17708368599414825, 0.05129425227642059, -0.16614845395088196, -0.1252318173646927, 0.0784161388874054, 0.1380375325679779, 0.018327131867408752, -0.09694088250398636, -0.08758850395679474, 0.03048521839082241, 0.08192328363656998, 0.023470936343073845, -0.016510464251041412, -0.041528064757585526, -0.03077283501625061, 0.012949516996741295, 0.016222847625613213, 0.031888075172901154, 0.016690464690327644, -0.010431419126689434, -0.03802092373371124, -0.03802092373371124, 0.004532379563897848, 0.03329093009233475, 0.06672567129135132, 0.0363304503262043, -0.08571802824735641, -0.1705908328294754, 0.010377613827586174, 0.22478079795837402, 0.10974659025669098, -0.16006940603256226, -0.20075222849845886, 0.010611423291265965, 0.14294752478599548, 0.11348754167556763, -0.0471394881606102, -0

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreT

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    "./",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 51094
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 1995


Epoch,Training Loss,Validation Loss,Accuracy
0,0.670500,0.555901,0.939688
1,0.294300,0.181935,0.975581
2,0.204000,0.122492,0.979112
3,0.176300,0.096331,0.981759
4,0.115100,0.092767,0.981318


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
Saving model checkpoint to ./checkpoint-399
Configuration saved in ./checkpoint-399/config.json
Model weights saved in ./checkpoint-399/pytorch_model.bin
Feature extractor saved in ./checkpoint-399/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
Saving model checkpoint to ./checkpoint-798
Configuration saved in ./checkpoint-798/config.json
Model weights saved in ./checkpoint-798/pytorch_model.bin
Feature extractor saved in ./checkpoint-798/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
Saving model checkpoint to ./checkpoint-1197
Configuration saved in ./checkpoint-1197/config.json
Model weights saved in ./checkpoint-1197/pytorch_model.bin
Feature extractor saved in ./checkpoint-1197/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
Saving model checkpoint to ./checkpoint-159

TrainOutput(global_step=1995, training_loss=0.4604133147643622, metrics={'train_runtime': 6864.3921, 'train_samples_per_second': 37.217, 'train_steps_per_second': 0.291, 'total_flos': 2.31918157475328e+18, 'train_loss': 0.4604133147643622, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


{'eval_loss': 0.0963306576013565,
 'eval_accuracy': 0.981759340982642,
 'eval_runtime': 70.4199,
 'eval_samples_per_second': 96.535,
 'eval_steps_per_second': 3.025,
 'epoch': 5.0}

You can now upload the result of the training to the Hub, just execute this instruction:

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("anton-l/my-awesome-model")
model = AutoModelForAudioClassification.from_pretrained("anton-l/my-awesome-model")

```